# Compute and store geographically bins spectra  
 - average spectra computed in parquet_time_window_spectra.ipynb on 2°bins using this mean positions
 - store it in one zarr

In [1]:
import os

import numpy as np
import pandas as pd
import xarray as xr
import dask.dataframe as dd

# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
# import geopandas as gpd
# from shapely.geometry import Polygon

%matplotlib inline
from matplotlib import pyplot as plt

import drifters.utils as ut
import pynsitu as pin
from GDP_lib import root_dir

In [2]:
from dask.distributed import Client

if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster

    # cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    # cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=5, processes=5, walltime="04:00:00")
    w = cluster.scale(jobs=5)
    # from dask_jobqueue import PBSCluster
    # cluster = PBSCluster()
    # w = cluster.scale(jobs=8) # 5 for gps, 8 for argos
else:
    from dask.distributed import LocalCluster

    cluster = LocalCluster()

client = Client(cluster)
client

/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/dask_jobqueue/core.py:237: FutureWarning: extra has been renamed to worker_extra_args. You are still using it (even if only set to []; please also check config files). If you did not set worker_extra_args yet, extra will be respected for now, but it will be removed in a future release. If you already set worker_extra_args, extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home1/datahome/mdemol/.miniconda3/envs/mdenv/lib/python3.9/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home1/datahome/mdemol/.min

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.0.111:8787/status,
Dashboard: http://10.148.0.111:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.0.111:43882,Workers: 0
Dashboard: http://10.148.0.111:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


# Choose key 

In [3]:
key = "argos"  # 'gps' or 'argos'
key2 = "_corrected_90_9999"  # or '_corrected'

---

# Geographically binned

https://github.com/apatlpo/mit_equinox/blob/master/parcels/spectra_binned.ipynb

In [14]:
DF = {}
labels = [
    "ven",
    "aen",
    "vxy",
    "axy",
    "vxydiff",
    "xy",
    "err_lonlat",
    "err_lonlatm",
    "err_vevn",
    "avxy",
]
for l in labels:
    parquet = os.path.join(root_dir, key + "_" + l + "_spectra" + key2 + ".parquet")
    df = dd.read_parquet(parquet)
    df = df.rename(
        columns={
            i: float(i)
            for i in list(df.columns)
            if i not in ["lon", "lat", "id", "date"]
        }
    )
    DF[l] = df

In [15]:
DF["xy"].head()

,lon,lat,id,0.0,0.016666666666666666,0.03333333333333333,0.05,0.06666666666666667,0.08333333333333333,0.1,...,-0.15,-0.13333333333333333,-0.11666666666666667,-0.1,-0.08333333333333333,-0.06666666666666667,-0.05,-0.03333333333333333,-0.016666666666666666,date
112252.0,-37.563995,24.345945,36727.0,1.233470e+11,1.288541e+09,1.990933e+10,3.316851e+09,4.973916e+08,5.862846e+06,1.737843e+08,...,4.959079e+06,2.990604e+07,1.531545e+07,2.856952e+08,4.817661e+08,1.520299e+08,1.363517e+09,1.591061e+09,1.135286e+11,2012-10-21 04:00:00
112972.0,-38.001958,23.565256,36727.0,6.400104e+11,7.979128e+10,2.728473e+09,2.661587e+09,1.941179e+08,1.029498e+08,2.958108e+08,...,1.324246e+07,6.670999e+06,6.212226e+07,1.278509e+08,3.987307e+07,3.387933e+08,1.657978e+09,3.204981e+09,3.716031e+11,2012-11-20 04:00:00
113692.0,-38.948332,23.100361,36727.0,1.220302e+12,1.809925e+11,3.904875e+09,3.732275e+08,9.242429e+08,5.547173e+08,4.231600e+08,...,1.419413e+07,1.105424e+08,1.429269e+08,3.632118e+07,1.728297e+08,2.616386e+09,2.818447e+07,1.869536e+10,4.231627e+11,2012-12-20 04:00:00
114412.0,-39.941199,22.911843,36727.0,3.209069e+12,6.999516e+11,5.306800e+09,2.793859e+09,1.366300e+08,3.153692e+07,1.090136e+07,...,1.739581e+07,8.035369e+07,2.395732e+07,1.954990e+08,4.382389e+08,1.314140e+09,1.297292e+09,3.250042e+10,1.278537e+12,2013-01-19 04:00:00
115132.0,-40.420775,22.691940,36727.0,4.260704e+12,1.268102e+12,4.581270e+09,8.736834e+07,2.255859e+07,6.970933e+06,4.997699e+06,...,2.776335e+06,6.822774e+06,4.235296e+07,2.302754e+07,9.128547e+06,1.310302e+08,6.889532e+08,2.023347e+09,9.553090e+11,2013-02-18 04:00:00


In [15]:
# bin geographically
dl = 2
lon_bins = np.arange(
    -180.0, 180.0 + dl, dl
)  # CAUTION : add dl to upper bound (prevent from pb with last binning intervals)
lat_bins = np.arange(-90, 90 + dl, dl)

### Computing

In [16]:
DF_geo = {}
Ds = {}
for l in DF:
    # DF[l].drop(columns=['ven2'], axis=1, inplace=True)
    DF[l]["lon"] = (DF[l]["lon"] + 180) % 360 - 180
    DF[l]["lon_cut"] = DF[l]["lon"].map_partitions(pd.cut, bins=lon_bins)
    DF[l]["lat_cut"] = DF[l]["lat"].map_partitions(pd.cut, bins=lat_bins)
    DF_geo[l] = (
        DF[l].groupby(["lon_cut", "lat_cut"]).mean().compute()
    )  # MEAN SPECTRUM OVER ALL SEGMENT IN THE LON, LAT bins

    # Converting pandas dataframe in xarray dataset:
    index = pd.MultiIndex.from_arrays(
        [
            DF_geo[l].index.map(lambda v: v[0].mid),
            DF_geo[l].index.map(lambda v: v[1].mid),
        ],
        names=("lon_cut", "lat_cut"),
    )
    ds = (
        (
            pd.melt(
                DF_geo[l]
                .set_index(index)
                .drop(columns=["id", "lon", "lat"])
                .reset_index(),
                id_vars=[
                    "lon_cut",
                    "lat_cut",
                ],
                var_name="frequency",
            ).rename(
                columns={
                    "lon_cut": "lon_bins",
                    "lat_cut": "lat_bins",
                    "value": "E_" + l,
                }
            )
        )
        .to_xarray()
        .set_index(index=["lon_bins", "lat_bins", "frequency"])
        .unstack()
    )
    ds["frequency"] = ds["frequency"].astype(float)
    ds = ds.sortby("frequency")
    # ds = ds.chunk({'frequency': 100})
    Ds[l] = ds
    print(l)

# nb counts
dsc = (
    DF[l]
    .groupby(["lon_cut", "lat_cut"])
    .size()
    .compute()
    .to_frame("nb_geobins")
    .set_index(index)
    .reset_index()
    .to_xarray()
    .rename({"lon_cut": "lon_bins", "lat_cut": "lat_bins"})
    .set_index(index=["lon_bins", "lat_bins"])
    .unstack()
)

ven
aen
vxy
axy
vxydiff
xy
err_lonlat
err_lonlatm
err_vevn
avxy


In [17]:
ds = xr.merge(list(Ds.values()) + [dsc])

# set attrs
ds.E_xy.attrs = {
    "long_name": "Power density spectra X = x + jy",
    "units": r"$m^2/cpd$",
    "description": "LOWESS method",
}
ds.E_ven.attrs = {
    "long_name": "Power density spectra v = ve + jvn",
    "units": r"$m^2/s^2/cpd$",
    "description": "LOWESS method",
}
ds.E_vxy.attrs = {
    "long_name": "Power density spectra v = vx + jvy",
    "units": r"$m^2/s^2/cpd$",
    "description": "geoid method",
}
ds.E_vxydiff.attrs = {
    "long_name": "Power density spectra v = vx + jvy",
    "units": r"$m^2/s^2/cpd$",
    "description": "finite differentiation method",
}
ds.E_axy.attrs = {
    "long_name": "Power density spectra a = ax + jay",
    "units": r"$m^2/s^4/cpd$",
    "description": "from position",
}
ds.E_aen.attrs = {
    "long_name": "Power density spectra a = ae + jan",
    "units": r"$m^2/s^4/cpd$",
    "description": "from ve,vn",
}

ds.frequency.attrs = {"long_name": "frequency", "units": "cpd"}
ds.nb_geobins.attrs = {
    "long_name": "nb_geobins",
    "description": "number of T days time window spectra per bins",
}

In [18]:
ds["E_venw2"] = ds["E_ven"] * (ds["frequency"] * 2 * np.pi / 3600 / 24) ** 2
ds["E_vxyw2"] = ds["E_vxy"] * (ds["frequency"] * 2 * np.pi / 3600 / 24) ** 2

In [19]:
ds.E_venw2.attrs = {
    "long_name": "E_ven $  \omega^2$",
    "units": r"$m^2/s^4/cpd$",
    "description": "from position",
}
ds.E_vxyw2.attrs = {
    "long_name": "E_vxy $  \omega^2$",
    "units": r"$m^2/s^4/cpd$",
    "description": "from ve,vn",
}

In [19]:
ds

<xarray.Dataset>
Dimensions:        (lon_bins: 180, lat_bins: 90, frequency: 1440)
Coordinates:
  * lon_bins       (lon_bins) float64 -179.0 -177.0 -175.0 ... 175.0 177.0 179.0
  * lat_bins       (lat_bins) float64 -89.0 -87.0 -85.0 -83.0 ... 85.0 87.0 89.0
  * frequency      (frequency) float64 -12.0 -11.98 -11.97 ... 11.95 11.97 11.98
Data variables: (12/13)
    E_ven          (lon_bins, lat_bins, frequency) float64 nan nan ... nan nan
    E_aen          (lon_bins, lat_bins, frequency) float64 nan nan ... nan nan
    E_vxy          (lon_bins, lat_bins, frequency) float64 nan nan ... nan nan
    E_axy          (lon_bins, lat_bins, frequency) float64 nan nan ... nan nan
    E_vxydiff      (lon_bins, lat_bins, frequency) float64 nan nan ... nan nan
    E_xy           (lon_bins, lat_bins, frequency) float64 nan nan ... nan nan
    ...             ...
    E_err_lonlatm  (lon_bins, lat_bins, frequency) float64 nan nan ... nan nan
    E_err_vevn     (lon_bins, lat_bins, frequency) float64 nan nan ... nan nan
    E_avxy         (lon_bins, lat_bins, frequency) float64 nan nan ... nan nan
    nb_geobins     (lon_bins, lat_bins) int64 0 0 0 0 0 0 0 0 ... 1 0 0 1 0 0 0
    E_venw2        (lon_bins, lat_bins, frequency) float64 nan nan ... nan nan
    E_vxyw2        (lon_bins, lat_bins, frequency) float64 nan nan ... nan nan


## Store geo spectra

In [20]:
root_dir = "/home1/datawork/mdemol/GDP"
zarr = os.path.join(root_dir, key + f"_geospectra_{int(dl)}" + key2 + ".zarr")

In [21]:
zarr

'/home1/datawork/mdemol/GDP/argos_geospectra_2_corrected_90_9999.zarr'

In [22]:
ds.to_zarr(zarr, mode="w")

In [23]:
ds_reload = xr.open_zarr(zarr).persist()
ds_reload

<xarray.Dataset>
Dimensions:        (lon_bins: 180, lat_bins: 90, frequency: 1440)
Coordinates:
  * frequency      (frequency) float64 -12.0 -11.98 -11.97 ... 11.95 11.97 11.98
  * lat_bins       (lat_bins) float64 -89.0 -87.0 -85.0 -83.0 ... 85.0 87.0 89.0
  * lon_bins       (lon_bins) float64 -179.0 -177.0 -175.0 ... 175.0 177.0 179.0
Data variables: (12/13)
    E_aen          (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_avxy         (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_axy          (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_err_lonlat   (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_err_lonlatm  (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_err_vevn     (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    ...             ...
    E_venw2        (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_vxy          (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_vxydiff      (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_vxyw2        (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    E_xy           (lon_bins, lat_bins, frequency) float64 dask.array<chunksize=(23, 23, 360), meta=np.ndarray>
    nb_geobins     (lon_bins, lat_bins) int64 dask.array<chunksize=(180, 90), meta=np.ndarray>

In [25]:
cluster.close()